**Lab 2. image Classification & CNNs**
Dataset de marcas de coches construido por: Esteban Berumen

Este código utiliza la arquitectura de VGG16 para conseguir los mejores resultados posibles.

In [3]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

In [4]:
# Carga de datos de CSV
train_df = pd.read_csv('train_dataset.csv')
test_df = pd.read_csv('test_dataset.csv')

# Directorios de imágenes
base_dir = os.getcwd()
folder_Train = os.path.join(base_dir, 'Train')
folder_Test = os.path.join(base_dir, 'Test')

# Tamaño de las imágenes
img_size = (128, 128)


In [5]:
!unrar x Test.rar
!unrar x Train.rar

"unrar" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"unrar" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [6]:
# Función para cargar las imágenes y etiquetas

def load_images_from_df(df, folder):
    images = []
    labels = []
    for index, row in df.iterrows():
        class_name = row['target']
        img_path = os.path.join(folder, class_name, row['filename'])
        try:
            img = load_img(img_path, target_size=img_size)
            img_array = img_to_array(img)
            images.append(img_array)
            labels.append(class_name)
        except FileNotFoundError:
            print(f"El archivo no fue encontrado: {img_path}")
    return np.array(images), labels

# Cargar imágenes y etiquetas de entrenamiento y prueba
X_train, y_train = load_images_from_df(train_df, folder_Train)
X_test, y_test = load_images_from_df(test_df, folder_Test)

# Convertir etiquetas a valores numéricos y luego a formato one-hot
y_train = pd.Series(y_train)
y_test = pd.Series(y_test)
y_train, uniques = pd.factorize(y_train)
y_test = pd.Categorical(y_test, categories=uniques).codes  # Mismo orden de categorías

y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Normalización de los datos de entrada
X_train = X_train / 255.0
X_test = X_test / 255.0

# Imprimir formas de los datos
print(f"Medidas Datos Train: {X_train.shape}")
print(f"Medidas Target Train: {y_train.shape}")
print(f"Medidas Datos Test: {X_test.shape}")
print(f"Medidas Target Test: {y_test.shape}")

Medidas Datos Train: (600, 128, 128, 3)
Medidas Target Train: (600, 10)
Medidas Datos Test: (60, 128, 128, 3)
Medidas Target Test: (60, 10)


In [7]:
# Definir el modelo VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Congelar las capas del modelo base
for layer in base_model.layers:
    layer.trainable = False

for layer in base_model.layers[-8:]:  # Descongela las últimas 8 capas
    layer.trainable = True

In [8]:
from tensorflow.keras.regularizers import l2

x = Flatten()(base_model.output)
x = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x)
output = Dense(10, activation='softmax')(x)

In [9]:
from tensorflow.keras.layers import Dropout

x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)  # Desactiva el 50% de las neuronas de esta capa
output = Dense(10, activation='softmax')(x)

In [10]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

In [11]:
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
output = Dense(10, activation='softmax')(x)  
model = Model(inputs=base_model.input, outputs=output)

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
19/19 [==============================] - 41s 2s/step - loss: 2.2417 - accuracy: 0.1917 - val_loss: 1.7912 - val_accuracy: 0.3500
Epoch 2/10
19/19 [==============================] - 47s 3s/step - loss: 1.3293 - accuracy: 0.5617 - val_loss: 0.9807 - val_accuracy: 0.6333
Epoch 3/10
19/19 [==============================] - 58s 3s/step - loss: 0.5585 - accuracy: 0.8133 - val_loss: 1.1576 - val_accuracy: 0.5667
Epoch 4/10
19/19 [==============================] - 50s 3s/step - loss: 0.3015 - accuracy: 0.9083 - val_loss: 1.1012 - val_accuracy: 0.6167
Epoch 5/10
19/19 [==============================] - 64s 3s/step - loss: 0.1708 - accuracy: 0.9517 - val_loss: 0.9950 - val_accuracy: 0.6833
Epoch 6/10
19/19 [==============================] - 51s 3s/step - loss: 0.0349 - accuracy: 0.9917 - val_loss: 0.7830 - val_accuracy: 0.7500
Epoch 7/10
19/19 [==============================] - 63s 3s/step - loss: 0.0136 - accuracy: 0.9983 - val_loss: 1.0494 - val_accuracy: 0.7667
Epoch 8/10
19/19 [==

In [13]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Precisión en el conjunto de prueba: {test_accuracy * 100:.2f}%")

2/2 [==============================] - 3s 1s/step - loss: 0.7906 - accuracy: 0.8000
Precisión en el conjunto de prueba: 80.00%


In [14]:
import matplotlib.pyplot as plt 

def visualize_activations_from_test_set(model, X_test, layer_indices, num_images=5):
    layer_outputs = [model.layers[i].output for i in layer_indices]
    activation_model = Model(inputs=model.input, outputs=layer_outputs)

    activations = activation_model.predict(X_test[:num_images])  

    for layer_index, activation in zip(layer_indices, activations):
        num_filters = activation.shape[-1]
        cols = 8  
        rows = (num_filters + cols - 1) // cols  

        fig, axes = plt.subplots(rows, cols, figsize=(20, 2 * rows))
        fig.suptitle(f'Activations of Layer {model.layers[layer_index].name}', fontsize=16)

        for filter_index in range(num_filters):
            for img_index in range(num_images):
                ax_index = img_index * num_filters + filter_index
                if ax_index < rows * cols:  
                    ax = axes.flatten()[ax_index]
                    ax.imshow(activation[img_index, :, :, filter_index], cmap='viridis')
                    ax.axis('off')
                else:
                    break  
        plt.subplots_adjust(hspace=0.4, wspace=0.4)
        plt.show()

layer_indices = [4, 9, 13, 19]  
num_images_to_visualize = 3  
visualize_activations_from_test_set(model, X_test, layer_indices, num_images=num_images_to_visualize)


1/1 [==============================] - 1s 559ms/step


: 